#Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from os.path import join
import os
from tqdm import tqdm

In [ ]:
!nvidia-smi

Fri Sep 10 16:39:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

%matplotlib inline

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path

In [ ]:
import tensorflow as tf

#Loading JPG Data

##TRAIN DATA

In [ ]:
imageSize=200
train_dir = Path( "/content/drive/MyDrive/Cs412Projectdata/train_data/")

In [ ]:
train_dir.glob(r'**/*.jpg')


<generator object Path.glob at 0x7f7d04805ed0>

In [ ]:
filepath = list(train_dir.glob(r'**/*.jpg'))

In [ ]:
classes = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepath))

In [ ]:
df = pd.concat([pd.Series(filepath).astype(str), pd.Series(classes)], axis=1)
df.columns = ['Images', 'Image_label']

In [ ]:
sample_size = 100
df = df.groupby("Image_label").sample(n=sample_size)

In [ ]:
df

,Images,Image_label
4481,/content/drive/MyDrive/Cs412Projectdata/train_...,A
4488,/content/drive/MyDrive/Cs412Projectdata/train_...,A
4139,/content/drive/MyDrive/Cs412Projectdata/train_...,A
4172,/content/drive/MyDrive/Cs412Projectdata/train_...,A
4671,/content/drive/MyDrive/Cs412Projectdata/train_...,A
...,...,...
17185,/content/drive/MyDrive/Cs412Projectdata/train_...,space
17760,/content/drive/MyDrive/Cs412Projectdata/train_...,space
17749,/content/drive/MyDrive/Cs412Projectdata/train_...,space
17125,/content/drive/MyDrive/Cs412Projectdata/train_...,space


##TEST DATA

In [ ]:

test_dir = Path("/content/drive/MyDrive/Cs412Projectdata/test_data/")


In [ ]:
test_dir.glob(r'**/*.jpg')

<generator object Path.glob at 0x7ff610ebbe50>

In [ ]:
filepath_Test = list(test_dir.glob(r'**/*.jpg'))

In [ ]:
classes_test = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepath_Test))

In [ ]:
df_test = pd.concat([pd.Series(filepath_Test).astype(str), pd.Series(classes_test)], axis=1)
df_test.columns = ['Images', 'Image_label']

In [ ]:
df_test.shape

(7250, 2)

In [ ]:
# sample_size_test = 50
# df_test = df_test.head(100)
# df_test

#Splitting data and Image process

##TRAIN DATA

In [ ]:
train_new, valid = train_test_split(df, test_size = 0.2, random_state = 42)
print(f"train set shape: {train_new.shape}")

print(f"validation set shape: {valid.shape}")

train set shape: (2320, 2)
validation set shape: (580, 2)


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.)


In [ ]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train_new,
                                              x_col = 'Images', y_col ='Image_label',
                                              target_size = (200,200), batch_size = 16, 
                                              class_mode = 'categorical', shuffle = True,featurewise_center=True)
val_gen = train_datagen.flow_from_dataframe(valid, 
                                            target_size=(200,200), x_col = 'Images', y_col ='Image_label', 
                                            class_mode='categorical',
                                            batch_size= 16, shuffle=False ,featurewise_center=True )


Found 2320 validated image filenames belonging to 29 classes.
Found 580 validated image filenames belonging to 29 classes.


In [ ]:
train_gen.image_shape

(200, 200, 3)

##TEST DATA

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
test_gen = test_datagen.flow_from_dataframe(df_test,
                                           target_size = (200,200), x_col = 'Images', y_col ='Image_label',
                                            class_mode = 'categorical',
                                           batch_size = 16, shuffle = False,featurewise_center = True)

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

#Creating model and tuning

##Base Model

In [ ]:
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape = (200,200,3),
                                                     include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()     #transfer learning

## Model With Random Parametre

In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(29,activation="softmax"))

In [ ]:
base_model.summary()

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.1857 - val_accuracy: 0.9552

Epoch 1/5000
145/145 [==============================] - 1305s 9s/step - loss: 2.4476 - accuracy: 0.3458 - val_loss: 0.7682 - val_accuracy: 0.7759
Epoch 2/5000
145/145 [==============================] - 20s 136ms/step - loss: 0.5440 - accuracy: 0.8487 - val_loss: 0.4422 - val_accuracy: 0.8776
Epoch 3/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.2499 - accuracy: 0.9270 - val_loss: 0.3749 - val_accuracy: 0.8759
Epoch 4/5000
145/145 [==============================] - 20s 136ms/step - loss: 0.1605 - accuracy: 0.9508 - val_loss: 0.2838 - val_accuracy: 0.9069
Epoch 5/5000
145/145 [==============================] - 20s 136ms/step - loss: 0.0730 - accuracy: 0.9861 - val_loss: 0.2933 - val_accuracy: 0.9138
Epoch 6/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.0645 - accuracy: 0.9868 - val_loss: 0.3392 - val_accuracy: 0.8931
Epoch 7/5000
145/145 [==============================] - 20s 136ms/step - loss: 0.0394 - accuracy: 0.9954 - val_loss: 0.

## Optimization of Learning Rate

In [ ]:
model_onelayer_lr = Sequential()
model_onelayer_lr.add(base_model)
model_onelayer_lr.add(GlobalAveragePooling2D())
model_onelayer_lr.add(Dense(256,activation="relu"))
model_onelayer_lr.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_onelayer_lr.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.1), metrics=['accuracy'])
history_onelayer_lr = model_onelayer_lr.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 39.3147 - accuracy: 0.0407 - val_loss: 3.3921 - val_accuracy: 0.0448

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 26s 144ms/step - loss: 39.3147 - accuracy: 0.0407 - val_loss: 3.3921 - val_accuracy: 0.0448
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 3.4077 - accuracy: 0.0372 - val_loss: 3.3983 - val_accuracy: 0.0293
Epoch 3/5000
145/145 [==============================] - 20s 136ms/step - loss: 3.5118 - accuracy: 0.0425 - val_loss: 3.3922 - val_accuracy: 0.0276
Epoch 4/5000
145/145 [==============================] - 20s 134ms/step - loss: 3.4003 - accuracy: 0.0271 - val_loss: 3.4280 - val_accuracy: 0.0345
Epoch 5/5000
145/145 [==============================] - 19s 133ms/step - loss: 3.4125 - accuracy: 0.0333 - val_loss: 3.4203 - val_accuracy: 0.0293
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 3.3983 - accuracy: 0.0341 - val_loss: 3.3884 - val_accuracy: 0.0310
Epoch 7/5000
145/145 [==============================] - 19s 134ms/step - loss: 3.3927 - accuracy: 0.0337 - val_loss: 

In [ ]:
model_onelayer_lr2 = Sequential()
model_onelayer_lr2.add(base_model)
model_onelayer_lr2.add(GlobalAveragePooling2D())
model_onelayer_lr2.add(Dense(256,activation="relu"))
model_onelayer_lr2.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_onelayer_lr2.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
history_onelayer_lr2 = model_onelayer_lr2.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.3304 - accuracy: 0.9045 - val_loss: 0.5880 - val_accuracy: 0.8586

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 143ms/step - loss: 4.2189 - accuracy: 0.3287 - val_loss: 1.1331 - val_accuracy: 0.6586
Epoch 2/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.6991 - accuracy: 0.7652 - val_loss: 0.8570 - val_accuracy: 0.7379
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.4757 - accuracy: 0.8313 - val_loss: 0.5916 - val_accuracy: 0.8276
Epoch 4/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.2733 - accuracy: 0.9111 - val_loss: 0.4780 - val_accuracy: 0.8552
Epoch 5/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2218 - accuracy: 0.9195 - val_loss: 0.5528 - val_accuracy: 0.8241
Epoch 6/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.2424 - accuracy: 0.9263 - val_loss: 0.8465 - val_accuracy: 0.8017
Epoch 7/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.3304 - accuracy: 0.9045 - val_loss: 0

In [ ]:
model_onelayer_lr = Sequential()
model_onelayer_lr.add(base_model)
model_onelayer_lr.add(GlobalAveragePooling2D())
model_onelayer_lr.add(Dense(256,activation="relu"))
model_onelayer_lr.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_onelayer_lr.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history_onelayer_lr = model_onelayer_lr.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 4.6215e-04 - accuracy: 1.0000 - val_loss: 0.1689 - val_accuracy: 0.9603

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 142ms/step - loss: 2.2387 - accuracy: 0.4107 - val_loss: 0.6908 - val_accuracy: 0.8155
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.4095 - accuracy: 0.8962 - val_loss: 0.3810 - val_accuracy: 0.8914
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2069 - accuracy: 0.9528 - val_loss: 0.3633 - val_accuracy: 0.8931
Epoch 4/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1185 - accuracy: 0.9782 - val_loss: 0.4395 - val_accuracy: 0.8776
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0782 - accuracy: 0.9860 - val_loss: 0.2847 - val_accuracy: 0.9259
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0431 - accuracy: 0.9975 - val_loss: 0.2215 - val_accuracy: 0.9379
Epoch 7/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0204 - accuracy: 1.0000 - val_loss: 0

## Optimization of Layers. 

###With Zero Hidden Layer

In [ ]:
model_Zerolayer = Sequential()
model_Zerolayer.add(base_model)
model_Zerolayer.add(GlobalAveragePooling2D())
model_Zerolayer.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5 , restore_best_weights=True)
model_Zerolayer.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_Zerolayer = model_Zerolayer.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)
#loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.1609 - val_accuracy: 0.9569

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 144ms/step - loss: 2.5014 - accuracy: 0.3428 - val_loss: 0.9498 - val_accuracy: 0.7724
Epoch 2/5000
145/145 [==============================] - 20s 134ms/step - loss: 0.7098 - accuracy: 0.8667 - val_loss: 0.6338 - val_accuracy: 0.8741
Epoch 3/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.3984 - accuracy: 0.9262 - val_loss: 0.4629 - val_accuracy: 0.9052
Epoch 4/5000
145/145 [==============================] - 20s 136ms/step - loss: 0.2753 - accuracy: 0.9637 - val_loss: 0.3973 - val_accuracy: 0.9155
Epoch 5/5000
145/145 [==============================] - 20s 136ms/step - loss: 0.1835 - accuracy: 0.9880 - val_loss: 0.3648 - val_accuracy: 0.9138
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1491 - accuracy: 0.9907 - val_loss: 0.3035 - val_accuracy: 0.9362
Epoch 7/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.1133 - accuracy: 0.9967 - val_loss: 0

### With One Hidden Layer

####Without Dropout

In [ ]:
model_onelayer = Sequential()
model_onelayer.add(base_model)
model_onelayer.add(GlobalAveragePooling2D())
model_onelayer.add(Dense(256,activation="relu"))
model_onelayer.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayer.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayer = model_onelayer.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1538 - val_accuracy: 0.9621

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 26s 142ms/step - loss: 2.3346 - accuracy: 0.3945 - val_loss: 0.6856 - val_accuracy: 0.8103
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.4441 - accuracy: 0.8842 - val_loss: 0.4853 - val_accuracy: 0.8569
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2213 - accuracy: 0.9521 - val_loss: 0.3187 - val_accuracy: 0.9086
Epoch 4/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1137 - accuracy: 0.9793 - val_loss: 0.2974 - val_accuracy: 0.9172
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0689 - accuracy: 0.9895 - val_loss: 0.2174 - val_accuracy: 0.9379
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0350 - accuracy: 0.9998 - val_loss: 0.1959 - val_accuracy: 0.9534
Epoch 7/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0255 - accuracy: 0.9996 - val_loss: 0

In [ ]:
model_onelayerDropN = Sequential()
model_onelayerDropN.add(base_model)
model_onelayerDropN.add(GlobalAveragePooling2D())
model_onelayerDropN.add(Dense(512,activation="relu"))

model_onelayerDropN.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_onelayerDropN.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history_onelayerDropN = model_onelayerDropN.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.1553 - val_accuracy: 0.9569

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 142ms/step - loss: 2.1375 - accuracy: 0.4416 - val_loss: 0.7052 - val_accuracy: 0.7690
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.4068 - accuracy: 0.8799 - val_loss: 0.4180 - val_accuracy: 0.8810
Epoch 3/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1495 - accuracy: 0.9620 - val_loss: 0.3412 - val_accuracy: 0.9086
Epoch 4/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1121 - accuracy: 0.9757 - val_loss: 0.2557 - val_accuracy: 0.9224
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0492 - accuracy: 0.9932 - val_loss: 0.2517 - val_accuracy: 0.9259
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0232 - accuracy: 0.9986 - val_loss: 0.2252 - val_accuracy: 0.9293
Epoch 7/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.0316 - accuracy: 0.9932 - val_loss: 0

####Dropout Added

In [ ]:
model_onelayerDrop = Sequential()
model_onelayerDrop.add(base_model)
model_onelayerDrop.add(GlobalAveragePooling2D())
model_onelayerDrop.add(Dense(256,activation="relu"))
model_onelayerDrop.add(Dropout(0.2))
model_onelayerDrop.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayerDrop.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayerDrop = model_onelayerDrop.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0145 - accuracy: 0.9978 - val_loss: 0.1627 - val_accuracy: 0.9621

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 142ms/step - loss: 2.5139 - accuracy: 0.3430 - val_loss: 0.7735 - val_accuracy: 0.8138
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.6782 - accuracy: 0.8008 - val_loss: 0.4800 - val_accuracy: 0.8759
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.3584 - accuracy: 0.9027 - val_loss: 0.3532 - val_accuracy: 0.9034
Epoch 4/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.1946 - accuracy: 0.9549 - val_loss: 0.2924 - val_accuracy: 0.9362
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1447 - accuracy: 0.9629 - val_loss: 0.3050 - val_accuracy: 0.9052
Epoch 6/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.1241 - accuracy: 0.9722 - val_loss: 0.2035 - val_accuracy: 0.9448
Epoch 7/5000
145/145 [==============================] - 21s 143ms/step - loss: 0.0813 - accuracy: 0.9853 - val_loss: 0

In [ ]:
model_onelayerDrop = Sequential()
model_onelayerDrop.add(base_model)
model_onelayerDrop.add(GlobalAveragePooling2D())
model_onelayerDrop.add(Dropout(0.1))
model_onelayerDrop.add(Dense(256,activation="relu"))
model_onelayerDrop.add(Dropout(0.2))
model_onelayerDrop.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayerDrop.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayerDrop = model_onelayerDrop.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0408 - accuracy: 0.9896 - val_loss: 0.1866 - val_accuracy: 0.9534

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 142ms/step - loss: 2.5136 - accuracy: 0.3244 - val_loss: 0.7643 - val_accuracy: 0.7931
Epoch 2/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.6993 - accuracy: 0.7992 - val_loss: 0.5062 - val_accuracy: 0.8534
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.4057 - accuracy: 0.8834 - val_loss: 0.3847 - val_accuracy: 0.8879
Epoch 4/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2431 - accuracy: 0.9356 - val_loss: 0.3141 - val_accuracy: 0.9103
Epoch 5/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1904 - accuracy: 0.9481 - val_loss: 0.2840 - val_accuracy: 0.9259
Epoch 6/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1573 - accuracy: 0.9502 - val_loss: 0.2462 - val_accuracy: 0.9276
Epoch 7/5000
145/145 [==============================] - 19s 132ms/step - loss: 0.0984 - accuracy: 0.9782 - val_loss: 0

In [ ]:
model_onelayerDropN = Sequential()
model_onelayerDropN.add(base_model)
model_onelayerDropN.add(GlobalAveragePooling2D())
model_onelayerDropN.add(Dense(512,activation="relu"))
model_onelayerDrop.add(Dropout(0.2))
model_onelayerDropN.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_onelayerDropN.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history_onelayerDropN = model_onelayerDropN.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)
#accuracy: 1.0000 - val_loss: 0.1422 - val_accuracy: 0.9603

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 140ms/step - loss: 2.2478 - accuracy: 0.4090 - val_loss: 0.6991 - val_accuracy: 0.7776
Epoch 2/5000
145/145 [==============================] - 19s 132ms/step - loss: 0.3733 - accuracy: 0.8892 - val_loss: 0.3662 - val_accuracy: 0.8948
Epoch 3/5000
145/145 [==============================] - 19s 132ms/step - loss: 0.1550 - accuracy: 0.9624 - val_loss: 0.2644 - val_accuracy: 0.9345
Epoch 4/5000
145/145 [==============================] - 19s 131ms/step - loss: 0.0945 - accuracy: 0.9798 - val_loss: 0.2476 - val_accuracy: 0.9293
Epoch 5/5000
145/145 [==============================] - 19s 132ms/step - loss: 0.0731 - accuracy: 0.9845 - val_loss: 0.2021 - val_accuracy: 0.9466
Epoch 6/5000
145/145 [==============================] - 19s 132ms/step - loss: 0.0320 - accuracy: 0.9946 - val_loss: 0.1685 - val_accuracy: 0.9534
Epoch 7/5000
145/145 [==============================] - 19s 132ms/step - loss: 0.0133 - accuracy: 1.0000 - val_loss: 0

### With Two Hidden Layer

####Without Dropout

In [ ]:
model_twolayer = Sequential()
model_twolayer.add(base_model)
model_twolayer.add(GlobalAveragePooling2D())
model_twolayer.add(Dense(256,activation="relu"))
model_twolayer.add(Dense(128,activation="relu"))
model_twolayer.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_twolayer.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_twolayer = model_twolayer.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.1955 - val_accuracy: 0.9603

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 26s 142ms/step - loss: 2.3761 - accuracy: 0.3375 - val_loss: 0.8336 - val_accuracy: 0.7431
Epoch 2/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.5183 - accuracy: 0.8460 - val_loss: 0.4782 - val_accuracy: 0.8586
Epoch 3/5000
145/145 [==============================] - 20s 134ms/step - loss: 0.2447 - accuracy: 0.9305 - val_loss: 0.3563 - val_accuracy: 0.9069
Epoch 4/5000
145/145 [==============================] - 20s 134ms/step - loss: 0.1369 - accuracy: 0.9610 - val_loss: 0.2819 - val_accuracy: 0.9224
Epoch 5/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.0514 - accuracy: 0.9913 - val_loss: 0.2730 - val_accuracy: 0.9310
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0620 - accuracy: 0.9885 - val_loss: 0.3489 - val_accuracy: 0.8914
Epoch 7/5000
145/145 [==============================] - 20s 134ms/step - loss: 0.0524 - accuracy: 0.9847 - val_loss: 0

In [ ]:
model_twolayerDrop1 = Sequential()
model_twolayerDrop1.add(base_model)
model_twolayerDrop1.add(GlobalAveragePooling2D())
model_twolayerDrop1.add(Dense(512,activation="relu"))
model_twolayerDrop1.add(Dense(128,activation="relu"))
model_twolayerDrop1.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_twolayerDrop1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_twolayerDrop1 = model_twolayerDrop1.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)
#loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.1687 - val_accuracy: 0.9586

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 142ms/step - loss: 2.3802 - accuracy: 0.3472 - val_loss: 0.8927 - val_accuracy: 0.7690
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.5174 - accuracy: 0.8662 - val_loss: 0.4246 - val_accuracy: 0.8862
Epoch 3/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.2737 - accuracy: 0.9093 - val_loss: 0.3540 - val_accuracy: 0.8966
Epoch 4/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1060 - accuracy: 0.9722 - val_loss: 0.2859 - val_accuracy: 0.9052
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0629 - accuracy: 0.9838 - val_loss: 0.2924 - val_accuracy: 0.9172
Epoch 6/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.0473 - accuracy: 0.9923 - val_loss: 0.2453 - val_accuracy: 0.9224
Epoch 7/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0259 - accuracy: 0.9954 - val_loss: 0

####With Dropout

In [ ]:
model_twolayerDrop = Sequential()
model_twolayerDrop.add(base_model)
model_twolayerDrop.add(GlobalAveragePooling2D())
model_twolayerDrop.add(Dense(256,activation="relu"))
model_twolayerDrop.add(Dropout(0.2))
model_twolayerDrop.add(Dense(128,activation="relu"))
model_twolayerDrop.add(Dropout(0.1))
model_twolayerDrop.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_twolayerDrop.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_twolayerDrop = model_twolayerDrop.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0876 - accuracy: 0.9733 - val_loss: 0.2155 - val_accuracy: 0.9414

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 26s 143ms/step - loss: 2.8025 - accuracy: 0.2246 - val_loss: 0.9229 - val_accuracy: 0.7466
Epoch 2/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.9088 - accuracy: 0.7162 - val_loss: 0.5969 - val_accuracy: 0.8293
Epoch 3/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.5302 - accuracy: 0.8242 - val_loss: 0.4187 - val_accuracy: 0.8741
Epoch 4/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.3552 - accuracy: 0.8968 - val_loss: 0.3900 - val_accuracy: 0.8776
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.2499 - accuracy: 0.9246 - val_loss: 0.2653 - val_accuracy: 0.9207
Epoch 6/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.1846 - accuracy: 0.9329 - val_loss: 0.2433 - val_accuracy: 0.9138
Epoch 7/5000
145/145 [==============================] - 20s 135ms/step - loss: 0.1715 - accuracy: 0.9389 - val_loss: 0

In [ ]:
model_twolayerDrop1 = Sequential()
model_twolayerDrop1.add(base_model)
model_twolayerDrop1.add(GlobalAveragePooling2D())
model_twolayerDrop1.add(Dense(512,activation="relu"))
model_twolayerDrop1.add(Dropout(0.2))
model_twolayerDrop1.add(Dense(128,activation="relu"))
model_twolayerDrop1.add(Dropout(0.1))
model_twolayerDrop1.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_twolayerDrop1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_twolayerDrop1 = model_twolayerDrop1.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.1375 - accuracy: 0.9573 - val_loss: 0.2442 - val_accuracy: 0.9276

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 141ms/step - loss: 2.7122 - accuracy: 0.2688 - val_loss: 0.8996 - val_accuracy: 0.7431
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.7911 - accuracy: 0.7370 - val_loss: 0.5731 - val_accuracy: 0.8310
Epoch 3/5000
145/145 [==============================] - 20s 134ms/step - loss: 0.4955 - accuracy: 0.8367 - val_loss: 0.4100 - val_accuracy: 0.8672
Epoch 4/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2759 - accuracy: 0.9242 - val_loss: 0.3944 - val_accuracy: 0.8655
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.2315 - accuracy: 0.9332 - val_loss: 0.3898 - val_accuracy: 0.8672
Epoch 6/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1694 - accuracy: 0.9498 - val_loss: 0.3362 - val_accuracy: 0.8948
Epoch 7/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1375 - accuracy: 0.9573 - val_loss: 0

### With Three Hidden Layer

#### Without Dropout

In [ ]:
model_threelayer = Sequential()
model_threelayer.add(base_model)
model_threelayer.add(GlobalAveragePooling2D())
model_threelayer.add(Dense(256,activation="relu"))
model_threelayer.add(Dense(128,activation="relu"))
model_threelayer.add(Dense(64,activation="relu"))
model_threelayer.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_threelayer.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_threelayer = model_threelayer.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

 #loss: 0.0747 - accuracy: 0.9825 - val_loss: 0.2887 - val_accuracy: 0.9172

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 141ms/step - loss: 2.6167 - accuracy: 0.2761 - val_loss: 0.8811 - val_accuracy: 0.7276
Epoch 2/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.6539 - accuracy: 0.7968 - val_loss: 0.5434 - val_accuracy: 0.8431
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.3184 - accuracy: 0.8971 - val_loss: 0.4952 - val_accuracy: 0.8379
Epoch 4/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.2010 - accuracy: 0.9397 - val_loss: 0.3531 - val_accuracy: 0.8862
Epoch 5/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1047 - accuracy: 0.9696 - val_loss: 0.3669 - val_accuracy: 0.8707
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0747 - accuracy: 0.9825 - val_loss: 0.2887 - val_accuracy: 0.9172
Epoch 7/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.0410 - accuracy: 0.9926 - val_loss: 0

In [ ]:
model_threelayerDropN1 = Sequential()
model_threelayerDropN1.add(base_model)
model_threelayerDropN1.add(GlobalAveragePooling2D())
model_threelayerDropN1.add(Dense(512,activation="relu"))
model_threelayerDropN1.add(Dense(256,activation="relu"))
model_threelayerDropN1.add(Dense(64,activation="relu"))
model_threelayerDropN1.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_threelayerDropN1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_threelayerDropN1 = model_threelayerDropN1.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0554 - accuracy: 0.9803 - val_loss: 0.2849 - val_accuracy: 0.9259

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 26s 146ms/step - loss: 2.6172 - accuracy: 0.2735 - val_loss: 0.9048 - val_accuracy: 0.7310
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.5941 - accuracy: 0.8290 - val_loss: 0.6026 - val_accuracy: 0.8017
Epoch 3/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.2942 - accuracy: 0.9102 - val_loss: 0.5453 - val_accuracy: 0.8466
Epoch 4/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1854 - accuracy: 0.9422 - val_loss: 0.3214 - val_accuracy: 0.9069
Epoch 5/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.0937 - accuracy: 0.9751 - val_loss: 0.3077 - val_accuracy: 0.8966
Epoch 6/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.1073 - accuracy: 0.9667 - val_loss: 0.2728 - val_accuracy: 0.9259
Epoch 7/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.0295 - accuracy: 0.9915 - val_loss: 0

####With Droput

In [ ]:
model_threelayerDrop = Sequential()
model_threelayerDrop.add(base_model)
model_threelayerDrop.add(GlobalAveragePooling2D())
model_threelayerDrop.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))
model_threelayerDrop.add(Dense(128,activation="relu"))
model.add(Dropout(0.1))
model_threelayerDrop.add(Dense(64,activation="relu"))
model.add(Dropout(0.1))
model_threelayerDrop.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_threelayerDrop.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_threelayerDrop = model_threelayerDrop.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0427 - accuracy: 0.9887 - val_loss: 0.2719 - val_accuracy: 0.9345

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 142ms/step - loss: 2.7629 - accuracy: 0.2484 - val_loss: 1.0790 - val_accuracy: 0.6328
Epoch 2/5000
145/145 [==============================] - 19s 134ms/step - loss: 0.7022 - accuracy: 0.7842 - val_loss: 0.5994 - val_accuracy: 0.8121
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.3134 - accuracy: 0.9099 - val_loss: 0.4712 - val_accuracy: 0.8603
Epoch 4/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2041 - accuracy: 0.9334 - val_loss: 0.3176 - val_accuracy: 0.9052
Epoch 5/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1004 - accuracy: 0.9786 - val_loss: 0.2832 - val_accuracy: 0.9172
Epoch 6/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.0646 - accuracy: 0.9846 - val_loss: 0.4092 - val_accuracy: 0.8793
Epoch 7/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.0843 - accuracy: 0.9734 - val_loss: 0

In [ ]:
model_threelayerDropN1 = Sequential()
model_threelayerDropN1.add(base_model)
model_threelayerDropN1.add(GlobalAveragePooling2D())
model_threelayerDropN1.add(Dense(512,activation="relu"))
model_threelayerDropN1.add(Dropout(0.2))
model_threelayerDropN1.add(Dense(256,activation="relu"))
model_threelayerDropN1.add(Dropout(0.1))
model_threelayerDropN1.add(Dense(64,activation="relu"))
model_threelayerDropN1.add(Dropout(0.1))
model_threelayerDropN1.add(Dense(29,activation="softmax"))

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_threelayerDropN1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_threelayerDropN1 = model_threelayerDropN1.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0868 - accuracy: 0.9687 - val_loss: 0.2340 - val_accuracy: 0.9397

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
145/145 [==============================] - 25s 141ms/step - loss: 3.0267 - accuracy: 0.1616 - val_loss: 1.4087 - val_accuracy: 0.5828
Epoch 2/5000
145/145 [==============================] - 19s 133ms/step - loss: 1.2793 - accuracy: 0.5930 - val_loss: 0.6971 - val_accuracy: 0.7810
Epoch 3/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.7450 - accuracy: 0.7453 - val_loss: 0.5662 - val_accuracy: 0.8052
Epoch 4/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.4998 - accuracy: 0.8279 - val_loss: 0.4193 - val_accuracy: 0.8655
Epoch 5/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.3170 - accuracy: 0.8997 - val_loss: 0.3403 - val_accuracy: 0.8966
Epoch 6/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.2707 - accuracy: 0.9116 - val_loss: 0.4299 - val_accuracy: 0.8845
Epoch 7/5000
145/145 [==============================] - 19s 133ms/step - loss: 0.1933 - accuracy: 0.9369 - val_loss: 0

##Activation function

In [ ]:
model_onelayer = Sequential()
model_onelayer.add(base_model)
model_onelayer.add(GlobalAveragePooling2D())
model_onelayer.add(Dense(256,activation="relu"))
model_onelayer.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayer.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayer = model_onelayer.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.1626 - val_accuracy: 0.9603

In [ ]:
model_onelayera = Sequential()
model_onelayera.add(base_model)
model_onelayera.add(GlobalAveragePooling2D())
model_onelayera.add(Dense(256,activation="sigmoid"))
model_onelayera.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayera.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayera = model_onelayera.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.1211 - val_accuracy: 0.9690


In [ ]:
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.1)

model_onelayera = Sequential()
model_onelayera.add(base_model)
model_onelayera.add(GlobalAveragePooling2D())
model_onelayera.add(Dense(256,activation=lrelu))
model_onelayera.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayera.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayera = model_onelayera.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)
#loss: 5.6412e-04 - accuracy: 1.0000 - val_loss: 0.1417 - val_accuracy: 0.9690

#Final Model

In [ ]:
model_onelayera = Sequential()
model_onelayera.add(base_model)
model_onelayera.add(GlobalAveragePooling2D())
model_onelayera.add(Dense(256,activation="sigmoid"))
model_onelayera.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_onelayera.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayera = model_onelayera.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

#accuracy: 1.0000 - val_loss: 0.1424 - val_accuracy: 0.9655

#Prediction

In [ ]:
predictions = model.predict(test_gen)
predictions

array([[5.50311673e-15, 1.10757009e-16, 2.30155641e-34, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.53641610e-27, 1.00000000e+00, 1.51939635e-28, ...,
        3.36206767e-35, 0.00000000e+00, 0.00000000e+00],
       [6.36826320e-21, 1.79483006e-17, 4.32728397e-13, ...,
        1.56764143e-10, 7.16260384e-12, 5.38871012e-17],
       ...,
       [1.01676473e-35, 4.23690326e-37, 1.11356156e-29, ...,
        1.25388419e-22, 5.35303631e-26, 1.65880233e-30],
       [0.00000000e+00, 5.85805541e-31, 9.99999762e-01, ...,
        2.65735782e-16, 2.04538674e-31, 4.64882017e-22],
       [7.28972037e-23, 7.81090749e-26, 8.22732837e-20, ...,
        8.84681252e-23, 4.06047825e-18, 4.47299201e-24]], dtype=float32)

In [ ]:
preds = np.argmax(predictions, axis=1 )
labels = train_gen.class_indices
labels


In [ ]:

labels = dict((v,k) for k,v in labels.items())
preds = [labels[k] for k in preds]

#Uploading

In [ ]:

new = []
for each in test_gen.filenames:
  a = each[66::]
  a = int(a.strip(".jpg"))
  new.append(a)

In [ ]:

last = pd.DataFrame({'Id':new, 'Prediction':preds} )

last = last.sort_values("Id")

In [ ]:
last.to_csv("submission.csv" , index = False)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(smp.Image_label, preds)

0.98